## About this notebook:

This notebook calculates a set of transformations based on segmented masks.

Input:
- pathway to a directory containing segmented labels
- pathway to a directory containing data frames
- list of wells to process
- downscale factor (1 - no downscaling, 2 - will make images 4 times smaller)

Output:
- a set of transforms for each well saved as pkl files

Downscale factor allows for calculations of transformations on smaller images (for huge images StackReg may run out of RAM - downscaling prevents solves the issue).

Optionally you can visualize alignment on downscaled images (visualization uses Napari).

## Fill in info about the experiment to process

In [3]:
# pathway to a directory with segmented masks for alignment (ex. im_segmented)
path_labels = r''

# pathway to a directory with data frames (ex. df)
path_df = r''

# list of wells to be processed (usually names as 'A3')
well_list = [''] 

# pathway to save transformations (ex. df)
path_save = r''

# specify a downscaling factor for the alignment
downscale_factor = 1 

## Prepare for processing

In [16]:
import os
import pickle

import pandas as pd
import numpy as np

from skimage import transform
from skimage.transform import downscale_local_mean

from pystackreg import StackReg
import pystackreg

In [14]:
def read_labels(path_labels,myWell):
    
    labels_list = os.listdir(os.path.join(path_labels,myWell))
    labels_list.sort()

    labels_im_list = []
    for lab_im_name in labels_list:

        lab_im = plt.imread(os.path.join(labels_dir,lab_im_name))
        labels_im_list.append(lab_im)

    labels_im = np.array(labels_im_list)
    
    return labels_im

def rescale_transforms(tmat_org,downscale_factor):
     
    # rescale transformation
    if downscale_factor != 1:
        
        tmat = []

        for tranform_matrix in tmat_org:

            eu_transform_small = transform.EuclideanTransform(tranform_matrix)

            eu_transform = transform.EuclideanTransform(translation = eu_transform_small.translation * downscale_factor,
                                                        rotation = eu_transform_small.rotation)

            tmat.append(eu_transform)
            
    else:
        tmat = tmat_org
            
    return tmat

def find_transformation(labels_im,downscale_factor = 1):

    # resize the image
    labels_small = labels_im>0
    labels_small = downscale_local_mean(labels_small,(1,downscale_factor,downscale_factor))
    
    # find transformation
    tf = StackReg.RIGID_BODY
    sr = StackReg(tf)
    
    tmat_small = sr.register_stack(labels_small, axis=0, reference='first', verbose=True)
    
    tmat = rescale_transforms(tmat_small,downscale_factor)
            
    return tmat,tmat_small

def apply_transforms_set(tmat,movie):
    
    res = []
    
    for index,tranform_matrix in enumerate(tmat):
    
        eu_transform = transform.EuclideanTransform(tranform_matrix)

        # if you want to check only transformation without rotation
        #eu_transform_small = transform.EuclideanTransform(translation = eu_transform_small.translation, rotation = 0)

        temp = transform.warp(movie[index,:,:],eu_transform,output_shape=movie[index].shape)

        res.append(temp)

    res = np.array(res)
    
    return res

## Process the wells

In [ ]:
for myWell in well_list:
    
    print(f'Processing well {myWell}.')
    
    # read in the data
    myData = pd.read_pickle(os.path.join(path_df,f'df_{myWell}.pkl'))
    
    # read in all the labels
    labels_im = read_labels(path_labels,myWell)
    
    # check if you have an expected number of files to align
    print(f'Number of label images {labels_im.shape[0]}')
    print(f'Number of unique rounds {len(set(myData.alignRound))}')
    
    # find transformation
    tmat,tmat_small = find_transformation(labels_im,downscale_factor = downscale_factor)
    
    # save transformations
    save_file_path = os.path.join(path_save,df_name.replace('df','tmat')) 
    pickle.dump(tmat, open(save_file_path, "wb"))
 

## Optional - test transformations

In [ ]:
import napari

In [18]:
# test transformations

# choose on which well to perform tests 
myWell = 'A1'

# read in images
labels_im = read_labels(path_labels,myWell)

# read in transformation matrix
tmat_list = pickle.load(open(os.path.join(path_save,f'tmat_{myWell}.pkl'), "rb"))
tmat_small = rescale_transforms(tmat_list,1/downscale_factor)

labels_small = downscale_local_mean(labels_im,(1,downscale_factor,downscale_factor))
labels_small_aligned = apply_transforms_set(tmat_small,labels_small)

viewer = napari.Viewer()
viewer.add_image(labels_small[0],blending ='additive',colormap='gray')
viewer.add_image(labels_small,blending ='additive',colormap='red')
viewer.add_image(labels_small_aligned,blending ='additive',colormap='green')

C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


<Image layer 'labels_small_aligned' at 0x2237f80f0c8>